In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/file/d/1Zf92g4V0tpRy_07ErJj7yZBFA714NBLx/view?usp=sharing'
id='1Zf92g4V0tpRy_07ErJj7yZBFA714NBLx'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('movies_genres.csv')  
data = pd.read_csv('movies_genres.csv', sep='\t')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data.head()

,title,plot,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Game-Show,History,Horror,Lifestyle,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,"""#7DaysLater"" (2013)",#7dayslater is an interactive comedy series f...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"""#BlackLove"" (2015) {Crash the Party (#1.9)}","With just one week left in the workshops, the...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,"""#BlackLove"" (2015) {Making Lemonade Out of Le...",All of the women start making strides towards...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,"""#BlackLove"" (2015) {Miss Independent (#1.5)}",All five of these women are independent and s...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,"""#BlackLove"" (2015) {Sealing the Deal (#1.10)}",Despite having gone through a life changing p...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [ ]:
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten,Dropout, Embedding

Using TensorFlow backend.


In [ ]:
genres = list(data.drop(['title','plot'],axis=1).columns.values)

In [ ]:
x_list = data['plot'].values
x_list=np.array(x_list)
len(x_list)

117352

In [ ]:
import nltk
nltk.download('stopwords')
from tqdm import tqdm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
for i in tqdm(range(len(x_list))):
    wor = x_list[i].split()
    words = [w for w in wor if w not in stopwords.words('english')]
    string = ' '.join(words)
    x_list[i] = string

100%|██████████| 117352/117352 [21:45<00:00, 89.91it/s] 


In [ ]:
y_list = data.drop(['title', 'plot'],axis=1)
y_list=y_list.to_numpy()
len(y_list)

117352

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_list, y_list, test_size=0.30, random_state=42)

In [ ]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', split=" ")
tokenizer.fit_on_texts(X_train)
x_train = tokenizer.texts_to_sequences(X_train)
x_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
x_train = pad_sequences(x_train,maxlen=500, padding='pre')   

In [ ]:
x_test = pad_sequences(x_test,maxlen=500, padding='pre')   

In [ ]:
x_test=np.array(x_test)

In [ ]:
n_feat = len(tokenizer.word_index)+1
labels=len(y_list[0])
n_feat

144707

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=n_feat, output_dim=100, input_length=500))
model.add(Dropout(0.3))
model.add(LSTM(64, return_sequences = True))
model.add(Dropout(0.3))
model.add(LSTM(64))
model.add(Dropout(0.3))
model.add(Dense(labels, activation = 'sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 100)          14470700  
_________________________________________________________________
dropout_1 (Dropout)          (None, 500, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 500, 64)           42240     
_________________________________________________________________
dropout_2 (Dropout)          (None, 500, 64)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 28)               

In [ ]:
h = model.fit(x_train,y_train, epochs=5,verbose=1, validation_data=(x_test,y_test))

Train on 82146 samples, validate on 35206 samples
Epoch 1/5
82146/82146 [==============================] - 1378s 17ms/step - loss: 0.1807 - accuracy: 0.9336 - val_loss: 0.1546 - val_accuracy: 0.9407
Epoch 2/5
82146/82146 [==============================] - 1249s 15ms/step - loss: 0.1392 - accuracy: 0.9476 - val_loss: 0.1363 - val_accuracy: 0.9486
Epoch 3/5
82146/82146 [==============================] - 1252s 15ms/step - loss: 0.1136 - accuracy: 0.9575 - val_loss: 0.1264 - val_accuracy: 0.9532
Epoch 4/5
82146/82146 [==============================] - 1253s 15ms/step - loss: 0.0934 - accuracy: 0.9650 - val_loss: 0.1221 - val_accuracy: 0.9563
Epoch 5/5
82146/82146 [==============================] - 1252s 15ms/step - loss: 0.0770 - accuracy: 0.9713 - val_loss: 0.1272 - val_accuracy: 0.9580


In [ ]:
preds=model.predict(x_test)
for i in range(len(preds)):
  print(np.argmax(preds[i]))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(np.argmax(y_test,axis=1), preds)

0.515565528603079

In [ ]:
X_test[300]

'People Know follows group friends New York City answers question happens group center falls out. Whose side choose? How far willing go achieve success? What role fidelity play relationships? Watch friends rise fall dream fail.'

In [ ]:
np.argmax(preds[100])
top3=np.argsort(preds[300])[:-4:-1]
top3

array([ 8,  5, 20])

In [ ]:
genres[5]

'Comedy'

In [ ]:
y_test[300]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [ ]:
genres

['Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Game-Show',
 'History',
 'Horror',
 'Lifestyle',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western']